# Voice Cloning

In [1]:
import speech_recognition as sr
import os
import glob
import wave

In [2]:
import argparse
import os
from pathlib import Path

import librosa
import numpy as np
import soundfile as sf
import torch

from encoder import inference as encoder
from encoder.params_model import model_embedding_size as speaker_embedding_size
from synthesizer.inference import Synthesizer
from utils.argutils import print_args
from utils.default_models import ensure_default_models
from vocoder import inference as vocoder

C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


## 1. Voice to Text Function

In [3]:
Folder_orig="data_wav/Orignal"
Folder_dest="data_wav/Fake"

In [4]:
def voice_to_text(voice):
    r = sr.Recognizer()
    with sr.AudioFile(str(voice)) as source:
        r.adjust_for_ambient_noise(source)

        audio = r.listen(source)

        try:
            text=r.recognize_google(audio)
            return(text)

        except Exception as e:
            print("Error {} : ".format(e) )

## 2. Text to Voice Function

In [5]:
# Hide GPUs from Pytorch to force CPU processing
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("Running a test of your configuration...\n")

if torch.cuda.is_available():
    device_id = torch.cuda.current_device()
    gpu_properties = torch.cuda.get_device_properties(device_id)
    ## Print some environment information (for debugging purposes)
    print("Found %d GPUs available. Using GPU %d (%s) of compute capability %d.%d with "
        "%.1fGb total memory.\n" %
        (torch.cuda.device_count(),
        device_id,
        gpu_properties.name,
        gpu_properties.major,
        gpu_properties.minor,
        gpu_properties.total_memory / 1e9))
else:
    print("Using CPU for inference.\n")

## Load the models one by one.
ensure_default_models(Path("saved_models"))
encoder.load_model(Path("saved_models/default/encoder.pt"))
synthesizer = Synthesizer("saved_models/default/synthesizer.pt")
vocoder.load_model(Path("saved_models/default/vocoder.pt"))

Running a test of your configuration...

Using CPU for inference.

Loaded encoder "encoder.pt" trained to step 1564501
Synthesizer using device: cpu
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at saved_models\default\vocoder.pt


In [6]:
def text_to_voice(sample_voice,audio_text,dest_path):
    try:
        
        in_fpath = sample_voice

        preprocessed_wav = encoder.preprocess_wav(in_fpath)

        original_wav, sampling_rate = librosa.load(str(in_fpath))
        preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)

        embed = encoder.embed_utterance(preprocessed_wav)

        text = audio_text

        # The synthesizer works in batch, so you need to put your data in a list or numpy array
        texts = [text]
        embeds = [embed]
        
        specs = synthesizer.synthesize_spectrograms(texts, embeds)
        spec = specs[0]

        generated_wav = vocoder.infer_waveform(spec)

        generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")

        # Trim excess silences to compensate for gaps in spectrograms (issue #53)
        generated_wav = encoder.preprocess_wav(generated_wav)

        # Save it on the disk
        filename = dest_path
        print(generated_wav.dtype)
        sf.write(filename, generated_wav.astype(np.float32), synthesizer.sample_rate)
        print("\nSaved output as %s\n\n" % filename)


    except Exception as e:
        print("Caught exception: %s" % repr(e))

## 3. Voice to Voice

In [7]:
def voice_to_voice(orig_audio,sample_voice,dest_audio):
    text=voice_to_text(orig_audio)
    text_to_voice(sample_voice,text,dest_audio)

In [8]:
Text_folder_orig=glob.glob(Folder_orig+"/*")
for i, path in enumerate(Text_folder_orig):
    try:
        orig_audio=Path(path)
        if(i<len(Text_folder_orig)-1):
            sample_audio=Path(Text_folder_orig[i+1])
        else:
            sample_audio=Path(Text_folder_orig[0])

        dest_audio=Path(Folder_dest+"\\fake%02d.wav" % i)

        voice_to_voice(orig_audio,sample_audio,dest_audio)
        
    except Exception as e:
        print("Caught exception: %s" % repr(e))

Error  : 


C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:42: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, source_sr, sampling_rate)
C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:42: FutureWarning: Pass orig_sr=22050, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, source_sr, sampling_rate)
C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:58: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -1.0382825e-15
  2.7775261e-16  0.0000000e+00], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(


Trainable Parameters: 30.870M
Caught exception: AttributeError("'str' object has no attribute 'name'")


C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:42: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, source_sr, sampling_rate)
C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:58: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(



| Generating 1/1


Done.

{| ████████████████ 28500/28800 | Batch Size: 3 | Gen Rate: 2.0kHz | }float64

Saved output as data_wav\Fake\fake01.wav




...


| Generating 1/1


Done.

{| ████████████████ 28500/28800 | Batch Size: 3 | Gen Rate: 2.0kHz | }float64

Saved output as data_wav\Fake\fake108.wav



| Generating 1/1


Done.

{| ████████████████ 47500/48000 | Batch Size: 5 | Gen Rate: 3.1kHz | }float64

Saved output as data_wav\Fake\fake109.wav




C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:58: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.3317167e-06
  3.2735936e-04  0.0000000e+00], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(



| Generating 1/1


Done.

{| ████████████████ 19000/19200 | Batch Size: 2 | Gen Rate: 1.3kHz | }float64

Saved output as data_wav\Fake\fake110.wav



| Generating 1/1


Done.

{| ████████████████ 28500/28800 | Batch Size: 3 | Gen Rate: 2.0kHz | }float64

Saved output as data_wav\Fake\fake111.wav




C:\Users\patel\AI Resedency Prog\VoiceCloning\Real-Time-Voice-Cloning-master\encoder\audio.py:58: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -2.7216063e-05
  8.8879060e-06  0.0000000e+00], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(



| Generating 1/1


Done.

{| ████████████████ 38000/38400 | Batch Size: 4 | Gen Rate: 2.5kHz | }float64

Saved output as data_wav\Fake\fake112.wav


